In [14]:
import numpy as np
import math
import importlib
import sys, os
import matplotlib.pyplot as plt
import time
import random
from numba import njit
sys.path.append(os.path.abspath('../../lib'))
import pairs_tensor_constructor
import pairs_tensor_util
import tree_sampler_DFPT
from util import compute_node_relations, determine_mutation_pair_occurance_counts
from tree_util import convert_parents_to_adjmatrix, convert_parents_to_ancmatrix, convert_adjmatrix_to_ancmatrix, convert_ancmatrix_to_adjmatrix, calc_tree_llh

from common import Models, NUM_MODELS
from tree_plotter import plot_tree
from pairs_tensor_plotter import plot_raw_scores

In [15]:
def load_cellcoal_data(fn):
    with open(fn,'r') as f:
        header = f.readline()
        n_rows, n_cols = header.replace("\n","").split(" ")
        n_rows, n_cols = int(n_rows), int(n_cols)
        data = np.zeros((int(n_rows/2), n_cols),dtype=int)
        for i, row in enumerate(f.readlines()):
            entries = row.replace("\n","")
            if i%2 == 0:
                maternal = entries.split("  ")[1]
            else:
                paternal = entries.split("  ")[1]
                assert len(maternal) == len(paternal)
                assert len(maternal) == n_cols
                cell_data = np.zeros(n_cols)
                for j in range(n_cols):
                    if maternal[j]=="?" and paternal[j]=="?":
                        cell_data[j] = 3
                    elif maternal[j]=="1" and paternal[j]=="1":
                        cell_data[j] = 2
                    elif maternal[j]=="1" or paternal[j]=="1":
                        cell_data[j] = 1
                    elif maternal[j]=="0" and paternal[j]=="0":
                        cell_data[j] = 0
                data[int((i-1)/2),:] = cell_data
    return data

def calc_true_anc(true_data,type="mut"):
    n_mut = true_data.shape[0]
    pairwise_counts_11 = determine_mutation_pair_occurance_counts(true_data, [1,1])
    pairwise_counts_10 = determine_mutation_pair_occurance_counts(true_data, [1,0])
    pairwise_counts_01 = determine_mutation_pair_occurance_counts(true_data, [0,1])
    # pairwise_counts_00 = determine_mutation_pair_occurance_counts(true_data, [0,0])

    is_coclust = (pairwise_counts_11>0) & (pairwise_counts_10==0) & (pairwise_counts_01==0)
    is_anc = (pairwise_counts_11>0) & (pairwise_counts_10>0) & (pairwise_counts_01==0)
    # is_dec = (pairwise_counts_11>0) & (pairwise_counts_10==0) & (pairwise_counts_01>0)
    # is_branched = (pairwise_counts_11==0) & (pairwise_counts_10>0) & (pairwise_counts_01>0)

    anc_mat = np.zeros((n_mut,n_mut),dtype=int)
    anc_mat += is_anc
    anc_mat += is_coclust

    if type=="mut":
        full_anc_mat = np.zeros((n_mut+1,n_mut+1),dtype=int)
        full_anc_mat[0,:] = 1
        full_anc_mat[1:,1:] = anc_mat
        return full_anc_mat
    elif type=="clust":
        clust_anc_mat, clst_inds, mut_clust_ass = np.unique(anc_mat,axis=0, return_inverse=True, return_index=True)
        clust_anc_mat = clust_anc_mat[:,clst_inds]
        n_clust = clust_anc_mat.shape[0]
        full_anc_mat = np.zeros((n_clust+1,n_clust+1),dtype=int)
        full_anc_mat[0,:] = 1
        full_anc_mat[1:,1:] = clust_anc_mat
        mut_clust_ass += 1
        return full_anc_mat, mut_clust_ass
    else:
        raise ValueError("type must be either 'mut' or 'clust'")


In [16]:
#Load in some of the data
fn = "./data/full_haplotypes_dir/full_hap.0001"
true_fn = "./data/true_haplotypes_dir/true_hap.0001"
data = load_cellcoal_data(fn)
true_data = load_cellcoal_data(true_fn)

data = np.transpose(data[:-1,:])
true_data = np.transpose(true_data[:-1,:])


In [17]:
print("0s:", np.sum(data==0))
print("1s:", np.sum(data==1))
print("2s:", np.sum(data==2))
print("3s:", np.sum(data==3))

0s: 4531799
1s: 38082
2s: 67
3s: 430052


In [26]:
#Compare number of observed snvs to actual snvs

n_true_snvs_in_data = np.sum(np.sum(true_data==1,axis=1)>0)
n_snvs_in_data = np.sum(np.sum(data==1,axis=1)>=50)



In [19]:
#Keep just the actual snvs for now
# data = data[np.sum(true_data==1,axis=1)>0,:]
# true_data = true_data[np.sum(true_data==1,axis=1)>0,:]

# #Determine the ancestry matices from the true data
# true_mut_anc = calc_true_anc(true_data,type="mut")
# true_clst_anc, mut_clst_ass = calc_true_anc(true_data,type="clust")

# true_clst_adj = convert_ancmatrix_to_adjmatrix(true_clst_anc)

In [20]:
# f = plot_tree(true_clst_adj)